<a href="https://colab.research.google.com/github/seongkyu-lim/Graduation-thesis/blob/master/YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (Tesla T4)


In [ ]:
# mount gdrive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Inference
이미 trained된 모델인 yolov5x,s,m,l 중 성능이 가장 좋은 xlarge모델로 inference 해보았습니다.

In [ ]:
import torch

%ls

#절대경로로 읿력해주어야 파일 위치 인식한다.
!python /content/yolov5/detect.py --weight yolov5x.pt --img 1280 --conf 0.4 --save-conf --save-txt --save-crop --source /content/yolov5/data/images/train

CONTRIBUTING.md  Dockerfile  LICENSE    requirements.txt*  utils/
data/            export.py   models/    train.py           val.py
detect.py        hubconf.py  README.md  tutorial.ipynb
100% 755k/755k [00:00<00:00, 31.8MB/s]
detect: weights=['yolov5x.pt'], source=/content/yolov5/data/images/train, imgsz=[1280, 1280], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-438-g27a4736 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

100% 168M/168M [00:01<00:00, 93.1MB/s]

Fusing layers... 
Model Summary: 476 layers, 87730285 parameters, 0 gradients
image 1/45 /content/yolov5/data/images/train/frame0.jpg: 736x1280 9 persons, Done. (0.125s)
image 2/45 /content/yolov5/data/images/train/frame1.jpg: 736x1280 14 p

In [ ]:
# 이미지로 확인해보기. 
# 영상으로 저장되서 불가능.
Image(filename='runs/detect/exp/', width=600)

ValueError: ignored

In [ ]:
# inference된 영상 다운로드 받기.
from google.colab import files
files.download('./inference/output/ARS-CHEL-0.mp4')

# 또는 구글드라이브로 옮겨서 확인하기.

#Train

  train하기

##Challenge

- 양 팀을 다르게 인식해야한다.
- 심판과 공또한 인식해야한다.

##1.opencv를 활용한 영상 처리

In [ ]:
%pip install opencv-python
%pip install numpy
%pip install matplotlib

In [ ]:
# -*- coding: utf-8 -*-
__author__ = 'Seran'
 
import cv2
 
# 영상의 의미지를 연속적으로 캡쳐할 수 있게 하는 class
vidcap = cv2.VideoCapture('/content/drive/MyDrive/졸업논문/20210822-ARS-CHE-EPL_1-1080.mkv')
 
count = 0

while(vidcap.isOpened()):
    ret, image = vidcap.read()
 
    if(int(vidcap.get(1)) % 200 == 0):
        print('Saved frame number : ' + str(int(vidcap.get(1))))
        cv2.imwrite("/content/yolov5/data/images/train/frame%d.jpg" % count, image) # data/images/train에 저장.
        print('Saved frame%d.jpg' % count)
        count += 1
 
vidcap.release()

Saved frame number : 200
Saved frame0.jpg
Saved frame number : 400
Saved frame1.jpg
Saved frame number : 600
Saved frame2.jpg
Saved frame number : 800
Saved frame3.jpg
Saved frame number : 1000
Saved frame4.jpg
Saved frame number : 1200
Saved frame5.jpg
Saved frame number : 1400
Saved frame6.jpg
Saved frame number : 1600
Saved frame7.jpg
Saved frame number : 1800
Saved frame8.jpg
Saved frame number : 2000
Saved frame9.jpg
Saved frame number : 2200
Saved frame10.jpg
Saved frame number : 2400
Saved frame11.jpg
Saved frame number : 2600
Saved frame12.jpg
Saved frame number : 2800
Saved frame13.jpg
Saved frame number : 3000
Saved frame14.jpg
Saved frame number : 3200
Saved frame15.jpg
Saved frame number : 3400
Saved frame16.jpg
Saved frame number : 3600
Saved frame17.jpg
Saved frame number : 3800
Saved frame18.jpg
Saved frame number : 4000
Saved frame19.jpg
Saved frame number : 4200
Saved frame20.jpg
Saved frame number : 4400
Saved frame21.jpg
Saved frame number : 4600
Saved frame22.jpg
Sa

KeyboardInterrupt: ignored

In [ ]:
%rm -rf /content/yolov5/data/images/train


## 2.yolov5로 train 진행

In [ ]:
# Download COCO128
torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

In [ ]:

'''
# Tensorboard  (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train
'''
# Weights & Biases(wandb)  (optional)
%pip install -q wandb
import wandb
wandb.login()


     |████████████████████████████████| 1.7 MB 8.3 MB/s 
     |████████████████████████████████| 97 kB 8.8 MB/s 
     |████████████████████████████████| 170 kB 66.9 MB/s 
     |████████████████████████████████| 133 kB 76.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# dataset.yaml에 훈련, valid를 위한 데이터셋 저장 경로 정보 있음.
!python train.py --img 640 --batch 16 --epochs 3 --data dataset.yaml --weights yolov5x.pt --cache

train: weights=yolov5x.pt, cfg=, data=dataset.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-438-g27a4736 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5, shear

In [ ]:
%rm -rf /content/yolov5/data/labels/train 
%rm -rf /content/yiolov5/data/labels/valid